In [1]:
import numpy as np
import tensorflow as tf

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

In [2]:
dataset_path = os.listdir('PlantVillage')

print (dataset_path)  

print("Types of classes labels found: ", len(dataset_path))

['Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight']
Types of classes labels found:  3


In [3]:
class_labels = []

for item in dataset_path:
 
 all_classes = os.listdir('PlantVillage' + '/' +item)



 for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))


In [4]:
# Build a dataframe        
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
print(df.head())
print(df.tail())

                  Labels                                              image
0  Potato___Early_blight  dataset_path/Potato___Early_blight/001187a0-57...
1  Potato___Early_blight  dataset_path/Potato___Early_blight/002a55fb-7a...
2  Potato___Early_blight  dataset_path/Potato___Early_blight/009c8c31-f2...
3  Potato___Early_blight  dataset_path/Potato___Early_blight/00d8f10f-50...
4  Potato___Early_blight  dataset_path/Potato___Early_blight/0182e991-97...
                    Labels                                              image
2147  Potato___Late_blight  dataset_path/Potato___Late_blight/fe9d57f1-8e0...
2148  Potato___Late_blight  dataset_path/Potato___Late_blight/feefc118-443...
2149  Potato___Late_blight  dataset_path/Potato___Late_blight/ff03a125-445...
2150  Potato___Late_blight  dataset_path/Potato___Late_blight/ff7160f9-f8d...
2151  Potato___Late_blight  dataset_path/Potato___Late_blight/fff55a7e-2f8...


In [5]:
# Let's check how many samples for each category are present
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

Total number of images in the dataset:  2152
Labels
Potato___Early_blight    1000
Potato___Late_blight     1000
Potato___healthy          152
Name: count, dtype: int64


In [6]:
import cv2
path = 'PlantVillage/'
dataset_path = os.listdir('PlantVillage')

im_size = 224

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)  
    filenames = [i for i in os.listdir(data_path) ]
   
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [7]:
images = np.array(images)

images = images.astype('float32') / 255.0
images.shape

(2152, 224, 224, 3)

In [8]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df['Labels'].values
print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

['Potato___Early_blight' 'Potato___Early_blight' 'Potato___Early_blight'
 ... 'Potato___Late_blight' 'Potato___Late_blight' 'Potato___Late_blight']
[0 0 0 ... 1 1 1]


In [9]:
y=y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
print(Y[:5])
print(Y[35:])

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [10]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, Y, random_state=1)


train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.05, random_state=415)


print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(2044, 224, 224, 3)
(2044, 3)
(108, 224, 224, 3)
(108, 3)


In [11]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

NUM_CLASSES = 3
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)


inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))



outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

In [12]:
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )

model.summary()

hist = model.fit(train_x, train_y, epochs=20, verbose=2)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 3)                4053414   
                                                                 
Total params: 4,053,414
Trainable params: 4,011,391
Non-trainable params: 42,023
_________________________________________________________________
Epoch 1/20
64/64 - 511s - loss: 1.1672 - accuracy: 0.7593 - 511s/epoch - 8s/step
Epoch 2/20
64/64 - 461s - loss: 0.4086 - accuracy: 0.8699 - 461s/epoch - 7s/step
Epoch 3/20
64/64 - 434s - loss: 0.2171 - accuracy: 0.9183 - 434s/epoch - 7s/step
Epoch 4/20
64/64 - 433s - loss: 0.1793 - accuracy: 0.9428 - 433s/epoch - 7s/step
Epoch 5/20
64/64 - 433s - loss: 0.1116 - accuracy: 0.9643 - 433s/epoch - 7s/step
Epoch 6/20
64/64 - 434s - loss: 0.096